In [1]:
import tensorflow as tf
from google.cloud import bigquery
import numpy as np
import shutil
tf.__version__

'2.1.0'

In [0]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename,mode,batch_size =512):
  def decode_csv(row):
    columns = tf.decode_csv(row,record_defaults=DEFAULTS)
    features = dict(zip(CSV_COLUMNS, columns))
    features.pop("key")
    label = features.pop("fare_amount")
    return features , label

  #create a list of filenames that matches global dataset
  filenames_dataset = tf.data.Dataset.list_files(filename,shuffle=False)
  textline_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
  dataset = textline_dataset.map(decode_csv)


  if mode==tf.estimator.Modekeys.TRAIN:
    num_epochs = None
    dataset= dataset.shuffle(buffer_size = 10 * batch_size,seed =2)
  else:
    num_epochs = 1

  dataset = dataset.repeat(num_epochs).batch(batch_size)

  return dataset






In [0]:
def get_train_input_fn():
  return read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid_input_fn():
  return read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

In [0]:
INPUT_COLUMNS =[
                tf.feature_column.numeric_column("pickuplon"),
                tf.feature_column.numeric_column("pickuplat"),
                tf.feature_column.numeric_column("dropofflat"),
                tf.feature_column.numeric_column("dropofflon"),
                tf.feature_column.numeric_column("passengers"),
]

def add_more_features(feats):
  return feats

In [0]:
feature_cols = add_more_features(INPUT_COLUMNS)

In [6]:

#tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.LinearRegressor(
      feature_columns = feature_cols, model_dir = OUTDIR)
model.train(input_fn = get_train_input_fn, steps = 200)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.

AttributeError: ignored

In [0]:
metrics = model.evaluate(input_fn = get_valid_input_fn, steps = None)
print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))